# Explore the Hierarchical class

This notebook explores the base functionality of Hierarchical as this base functionality is introduced.

For now, Hierarchical is indeed instantiated as a subclass of Sequential, but only inherits the methods that specify how the layer architecture works. If more general architectures are added, the actual implementation will be moved up. Currently, this implementation already involves the general Model API in keras.

## Methods in Sequential

In [1]:
import keras
seq = keras.Sequential()

Using TensorFlow backend.
W0828 14:37:13.743058 140407269582656 deprecation_wrapper.py:119] From /home/sflippl/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [8]:
seq.add(keras.layers.Dense(10, input_shape=(4, )))

W0828 14:54:07.034281 140407269582656 deprecation_wrapper.py:119] From /home/sflippl/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 14:54:07.037168 140407269582656 deprecation_wrapper.py:119] From /home/sflippl/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [9]:
seq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________


Essentially, the thing we must add is a possibility to infer arbitrary states; essentially a state layer. Later, this will be ample opportunity to specify different kinds of state layers (think probabilistic, a la Srinivasan, etc.). Later, we may thus have more than two different modes of estimation, but for now, we have state and weight parameter estimation. This means that effectively, we have multiple sequential models -- which also means that we might as well just explicitly have an entirely new class.

The overall UI should enable state-to-state thinking. Conceptually, there are two levels of a predictive coding model: on one level, there are the state inference layers. On the other hand, each of these inferred states is being used to predict the state blow, the lowest state being actually observed. The supernetworks should be specified in a bottom-up fashion, whereas the subnetworks should be specified in a top-down fashion. When Hierarchical is initialized, it should thus be initialized in a top-down fashion.

The following syntax thus seems sensible:

Or, as a more complex example:

Pro: Hierarchical models can be built in a bottom up fashion.
Con: There's a somewhat unnecessary doubling of state estimation and state -- or at least the input shape there.
However, I can see certain advantages with that kind of verbosity, as well.

As a first step, I will implement the pure interface plus printing.


Revamp: There are essentially three different modi: adding the state tiers, adding the tier models, and adding the connections between the tiers.

In [35]:
seq = keras.Sequential()
seq.add(keras.layers.Dense(10, input_shape=(4, )))
seq.add(keras.layers.Activation('relu'))
seq.add(keras.layers.Dense(4))
seq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10)                50        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 44        
Total params: 94
Trainable params: 94
Non-trainable params: 0
_________________________________________________________________


Importantly, we separate the user interface from the implementation via the appropriate class methods. Though I have not looked at that yet, this should make an alternative backend to Tensorflow easier.

This kind of interface has now been achieved:

In [1]:
import predicode as pc
import keras
hpc = pc.Hierarchical()
hpc.add_tier(shape=(10, ))
hpc.add_tier(shape=(4, ))
hpc.predictor = keras.Sequential()
hpc.predictor.add(keras.layers.Dense(4, input_shape=(10, )))
hpc.summary()

Using TensorFlow backend.
W0830 13:16:59.213186 139939909711680 deprecation_wrapper.py:119] From /home/sflippl/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0830 13:16:59.229441 139939909711680 deprecation_wrapper.py:119] From /home/sflippl/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0830 13:16:59.235038 139939909711680 deprecation_wrapper.py:119] From /home/sflippl/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



# Tier 1: tier_1
## Connecting Predictor
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 44        
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________
## Connection State Prediction
(No state prediction defined.)
# Tier 0: tier_0


Now, we tackle the real meat of Hierarchical: the estimation.